In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

def interactive_B0_viewer(B0_maps, vmin=-75, vmax=75):
    views = {
        "Sagittal": lambda vol, idx: vol[idx, :, :],
        "Coronal": lambda vol, idx: vol[:, idx, :],
        "Axial": lambda vol, idx: vol[:, :, idx]
    }

    shape = B0_maps['default'].shape
    methods = list(B0_maps.keys())

    sag_slider = widgets.IntSlider(min=0, max=shape[0]-1, value=shape[0]//2, description='Sagittal')
    cor_slider = widgets.IntSlider(min=0, max=shape[1]-1, value=shape[1]//2, description='Coronal')
    axi_slider = widgets.IntSlider(min=0, max=shape[2]-1, value=shape[2]//2, description='Axial')

    sliders = widgets.HBox([sag_slider, cor_slider, axi_slider])

    output = widgets.Output()

    def plot_slices(sag, cor, axi):
        with output:
            output.clear_output(wait=True)
            slice_dict = {"Sagittal": sag, "Coronal": cor, "Axial": axi}
            fig, axes = plt.subplots(len(methods), len(views), figsize=(12, 3 * len(methods)))

            for i, method in enumerate(methods):
                B0 = B0_maps[method]
                for j, (view_name, slicer) in enumerate(views.items()):
                    idx = slice_dict[view_name]
                    img = slicer(B0, idx).T
                    ax = axes[i, j]
                    ax.imshow(img, cmap='jet', vmin=vmin, vmax=vmax, origin='lower')
                    ax.set_title(f"{method} - {view_name}")
                    ax.axis("off")
                    fig.colorbar(ax.images[0], ax=ax, fraction=0.046, pad=0.01)

            plt.tight_layout()
            plt.show()

    ui = widgets.interactive_output(plot_slices, {
        'sag': sag_slider,
        'cor': cor_slider,
        'axi': axi_slider
    })

    app = widgets.VBox([sliders, output])
    display(app)

    plot_slices(sag_slider.value, cor_slider.value, axi_slider.value)


In [ ]:
import nibabel as nib

B0_noshim = nib.load("Nifti/CY_PFC_nii/CY_PFC_deltaB0_hertz.nii.gz").get_fdata() 
B0_aftershim = nib.load("SF_shimming_test/Nifti/CY_PFC_test2_nii/CY_PFC_test2_deltaB0_hertz.nii.gz").get_fdata() 


B0_maps = {
    "default": B0_noshim,
    "shim": B0_noshim
}

interactive_B0_viewer(B0_maps, vmin=-2, vmax=2)


FileNotFoundError: No such file or no access: 'SF_shimming_test/Nifti/CY_PFC_nii/CY_PFC_deltaB0_hertz.nii.gz'